In [39]:
cd D:\Algorithmic Marketing\Assignment 3\recommenders

D:\Algorithmic Marketing\Assignment 3\recommenders


In [40]:
import sys
sys.path.append("../../")
import os
import cornac
import papermill as pm
import pandas as pd
from reco_utils.dataset import movielens
from reco_utils.dataset.python_splitters import python_random_split
from reco_utils.evaluation.python_evaluation import map_at_k, ndcg_at_k, precision_at_k, recall_at_k
from reco_utils.recommender.cornac.cornac_utils import predict_ranking
from reco_utils.common.timer import Timer
from reco_utils.common.constants import SEED

print("System version: {}".format(sys.version))
print("Cornac version: {}".format(cornac.__version__))

System version: 3.7.4 (default, Aug  9 2019, 18:34:13) [MSC v.1915 64 bit (AMD64)]
Cornac version: 1.7.1


In [41]:
df = pd.read_csv("D:/Algorithmic Marketing/Final Project/Dataset/Recommendation_data.csv")

In [42]:
df.head()

,userID,itemID,rating
0,17850,85123A,8
1,17850,71053,7
2,17850,84406B,8
3,17850,84029G,5
4,17850,84029E,9


In [43]:
ALIBABA_DATA_SIZE = '100k'
TOP_K = 10

# Model parameters
NUM_FACTORS = 200
NUM_EPOCHS = 100

In [44]:
train, test = python_random_split(df, 0.75)

In [45]:
train_set = cornac.data.Dataset.from_uir(train.itertuples(index=False), seed=SEED)

C:\Users\verle\Anaconda3\lib\site-packages\cornac\data\dataset.py:361: UserWarning: 27530 duplicated observations are removed!
  warnings.warn("%d duplicated observations are removed!" % dup_count)


In [46]:
train_set = cornac.data.Dataset.from_uir(train.itertuples(index=False), seed=SEED)

print('Number of users: {}'.format(train_set.num_users))
print('Number of items: {}'.format(train_set.num_items))

Number of users: 602
Number of items: 3202


In [47]:
bpr = cornac.models.BPR(
    k=NUM_FACTORS,
    max_iter=NUM_EPOCHS,
    learning_rate=0.01,
    lambda_reg=0.001,
    verbose=True,
    seed=SEED
)

In [48]:
with Timer() as t:
    bpr.fit(train_set)
print("Took {} seconds for training.".format(t))


Optimization finished!
Took 2.6919 seconds for training.


In [49]:
import pickle

In [52]:
pickle.dump( bpr, open( "lightGCN_model.p", "wb" ) )

In [53]:
os.listdir()

['.git',
 '.github',
 '.gitignore',
 'AUTHORS.md',
 'bpr_model.p',
 'contrib',
 'CONTRIBUTING.md',
 'docs',
 'examples',
 'GLOSSARY.md',
 'LICENSE',
 'lightGCN_model.p',
 'ligthGCN_model.p',
 'README.md',
 'reco_base.yaml',
 'reco_utils',
 'scenarios',
 'SECURITY.md',
 'SETUP.md',
 'setup.py',
 'tests',
 'tools']

In [55]:
import pickle
bpr = pickle.load( open( "lightGCN_model.p", "rb" ) )

In [56]:
with Timer() as t:
    all_predictions = predict_ranking(bpr, train, usercol='userID', itemcol='itemID', remove_seen=True)
print("Took {} seconds for prediction.".format(t))

Took 2.0572 seconds for prediction.


In [57]:
k = 10
eval_map = map_at_k(test, all_predictions, col_prediction='prediction', k=k)
eval_ndcg = ndcg_at_k(test, all_predictions, col_prediction='prediction', k=k)
eval_precision = precision_at_k(test, all_predictions, col_prediction='prediction', k=k)
eval_recall = recall_at_k(test, all_predictions, col_prediction='prediction', k=k)

print("MAP:\t%f" % eval_map,
      "NDCG:\t%f" % eval_ndcg,
      "Precision@K:\t%f" % eval_precision,
      "Recall@K:\t%f" % eval_recall, sep='\n')

MAP:	0.014445
NDCG:	0.058248
Precision@K:	0.045870
Recall@K:	0.035324


In [58]:
#689
101


101

In [59]:
user = input("Enter User ID")
item = input("Enter Item ID ")

df1 = pd.DataFrame(data={
    "userID":[int(user)],
    "itemID":[str(item)]
})

Enter User ID17850
Enter Item ID 71053


In [60]:
with Timer() as t:
    foo = predict_ranking(bpr, df1, usercol='userID', itemcol='itemID', remove_seen=True)
print("Took {} seconds for prediction.".format(t))

Took 2.0539 seconds for prediction.


In [61]:
foo["userID"] = foo["userID"] .apply(lambda x :  round(x))
foo["itemID"] = foo["itemID"] .apply(lambda x :  (x))

In [62]:
result = foo[foo["userID"]==int(user)].sort_values(by='prediction', ascending=False).head(10)

In [63]:
result = result.drop(columns=["prediction", "userID"], axis=1)

In [64]:
result.to_json()

'{"itemID":{"788096":"85123A","787911":"22423","788184":"84879","787789":"22086","788123":"21034","788326":"23084","788075":"22469","787774":"82494L","788148":"22865","788052":"21733"}}'